<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Meta_Learning_(Few_Shot_Learning).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable

class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.fc1 = nn.Linear(1, 1)
        self.fc2 = nn.Linear(1, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        return self.fc2(x)

def maml_update(model, loss_fn, data, target, lr=0.01):
    # Compute gradients for the task
    model.zero_grad()
    output = model(data)
    loss = loss_fn(output, target)
    loss.backward()
    # Update model weights
    with torch.no_grad():
        for param in model.parameters():
            param -= lr * param.grad
    return loss.item()

model = SimpleModel()
optimizer = optim.SGD(model.parameters(), lr=0.1)
loss_fn = nn.MSELoss()

# Training loop for MAML
for epoch in range(10):
    data = torch.randn(5, 1)  # Few-shot examples (5 examples)
    target = 2 * data + 1     # Target for linear regression

    # Inner loop: Adapt the model to a task
    loss = maml_update(model, loss_fn, data, target, lr=0.01)

    # Outer loop: Meta-optimization (MAML)
    optimizer.zero_grad()
    output = model(data)
    meta_loss = loss_fn(output, target)
    meta_loss.backward()
    optimizer.step()

    print(f'Epoch {epoch}, Loss: {meta_loss.item()}')